In [116]:
import pandas as pd
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import os
from sklearn.linear_model import RidgeCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np


In [117]:
# load data

prefex = '/Users/rjing/Desktop/Machine_Learning_Nonpoint_Source_Pollution/data/water_runoff_sample/Total_Phosphorus/'
#prefex = '/Users/rjing/Desktop/Machine_Learning_Nonpoint_Source_Pollution/data/water_runoff_sample/Total_Nitrogen/'
#prefex = '/Users/rjing/Desktop/Machine_Learning_Nonpoint_Source_Pollution/data/water_runoff_sample/Nitrate_Nitrogen/'
#prefex = '/Users/rjing/Desktop/Machine_Learning_Nonpoint_Source_Pollution/data/water_runoff_sample/Dissolved_Phosphorus/'
#prefex = '/Users/rjing/Desktop/Machine_Learning_Nonpoint_Source_Pollution/data/water_runoff_sample/Particulate_Phosphorus/'
#prefex = '/Users/rjing/Desktop/Machine_Learning_Nonpoint_Source_Pollution/data/water_runoff_sample/Ammoniacal_Nitrogen/'
#prefex = '/Users/rjing/Desktop/Machine_Learning_Nonpoint_Source_Pollution/data/water_leaching_sample_30cm/'

file_list = [f for f in os.listdir(prefex) if f.endswith(".csv")]
print(file_list)
    

['Water_sample_runoff_citrus_Total_Phosphorus.csv', 'Water_sample_runoff_corn_Total_Phosphorus.csv', 'Water_sample_runoff_rice_Total_Phosphorus.csv', 'Water_sample_runoff_vegetable_Total_Phosphorus.csv']


In [118]:
def Multiple_Linear_Regression_Combined(files, analysistype, prefex):
    """Performs multiple linear regression for multiple files and saves all plots in one large figure."""
    
    dependent_columns = ['NPK', 'PK', 'NK', 'CK', 'OF']
    num_files = len(files)
    num_vars = len(dependent_columns)

    # Create a large figure (rows = files, columns = dependent variables)
    fig, axes = plt.subplots(nrows=num_files, ncols=num_vars, figsize=(8 * num_vars, 8 * num_files))

    results_list = []

    for file_idx, file_name in enumerate(files):
        # Load dataset
        data = pd.read_csv(prefex + file_name, encoding='latin1')

        # Define independent variables (X)
        X = data[data.columns[6:]]
        X = X.drop(X.columns[1], axis=1)
        X = sm.add_constant(X)

        for var_idx, column in enumerate(dependent_columns):
            if column not in data:
                continue  # Skip if column is missing

            y = data[column].dropna()
            X_filtered, y_filtered = X.align(y, join="inner", axis=0)

            # Split the data
            X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.2, random_state=42)

            # Fit OLS model
            model = sm.OLS(y_train, X_train).fit()
            R2 = round(model.rsquared, 3)
            adj_R2 = round(model.rsquared_adj, 3)

            # Store results
            results_list.append({
                'File': file_name,
                'Dependent Variable': column,
                'R-squared': R2,
                'Adjusted R-squared': adj_R2
            })

            # **Scatter plot with trend line**
            ax = axes[file_idx, var_idx] if num_files > 1 else axes[var_idx]  # Handle single file case
            sns.scatterplot(x=model.fittedvalues, y=y_train, ax=ax, label="Actual Data", s=160, edgecolor='black', facecolor='#7209B7') 
            #sns.scatterplot(x=model.fittedvalues, y=y_train, ax=ax, label="Actual Data", s=160, edgecolor='black', facecolor='#4CC9F0') 
            #sns.scatterplot(x=model.fittedvalues, y=y_train, ax=ax, label="Actual Data", s=160, edgecolor='black', facecolor='#F72585') 
            #sns.scatterplot(x=model.fittedvalues, y=y_train, ax=ax, label="Actual Data", s=160, edgecolor='black', facecolor='#4361EE') 
            #sns.scatterplot(x=model.fittedvalues, y=y_train, ax=ax, label="Actual Data", s=160, edgecolor='black', facecolor='#008000') 
            #sns.scatterplot(x=model.fittedvalues, y=y_train, ax=ax, label="Actual Data", s=160, edgecolor='black', facecolor='#FFA500') 
            
            sns.lineplot(x=model.fittedvalues, y=model.fittedvalues, color='red', 
                         ax=ax, label="Trend Line", linewidth=3)
            

            ax.set_title(f"{column}{'-'}{file_name.split('.')[0].split('_')[3]} (R²={R2})", fontsize=20)
            ax.set_xlabel("Predicted Values", fontsize=18)
            ax.set_ylabel(f"Actual {column}", fontsize=18)
            ax.tick_params(axis='both', labelsize=16)
            ax.legend(fontsize=16)
            ax.grid()

    # Adjust layout
    plt.tight_layout()
    plt.suptitle("Scatter Plots & Trend Lines for" + " " + 
                 prefex.split('/')[-3].split('_')[0] + " " +
                 prefex.split('/')[-3].split('_')[1] + " " +
                 prefex.split('/')[-3].split('_')[2] + " " +
                 '(' +  analysistype + ')',
                 fontsize=26, y=1.02)

    # Save to a single PDF file
    pdf_filename = prefex + "Multiple_Linear_Regression_Results/" +"combined_regression_plots" + "_" + analysistype.split(' ')[0] + "_" + analysistype.split(' ')[1] +".pdf"
    with PdfPages(pdf_filename) as pdf:
        pdf.savefig(fig, bbox_inches='tight', dpi=400)
        plt.close(fig)
    

    # Save results to CSV
    results_df = pd.DataFrame(results_list)
    results_df.to_csv(prefex + "Multiple_Linear_Regression_Results/" + "combined_regression_results" + "_" + analysistype.split(' ')[0] + "_" + analysistype.split(' ')[1] +".csv", index=False)

    return results_df.to_numpy().tolist()


def main():
    selected_files = []
    # Filter files that match the "Total_Nitrogen" condition
    for file in file_list:
        selected_files.append(file)
        # Run regression for all selected files in one call
        if selected_files:
             Multiple_Linear_Regression_Combined(selected_files, file.split('.')[0].split('_')[-2] + " " + file.split('.')[0].split('_')[-1], prefex)

    print("Job done!")


if __name__=="__main__":
    main()


Job done!


In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.backends.backend_pdf import PdfPages
import scipy.stats as stats

def Multiple_Linear_Regression_Combined(files, analysistype, prefex):
    """Performs multiple linear regression, generates advanced plots, and saves results."""
    
    dependent_columns = ['NPK', 'PK', 'NK', 'CK', 'OF']
    num_files = len(files)
    
    results_list = []

    for file_name in files:
        # Load dataset
        data = pd.read_csv(prefex + file_name, encoding='latin1')

        # Define independent variables (X)
        X = data.iloc[:, 6:].drop(columns=[data.columns[7]])  # Drop second column as per your logic
        X = sm.add_constant(X)

        for column in dependent_columns:
            if column not in data:
                continue

            y = data[column].dropna()
            X_filtered, y_filtered = X.align(y, join="inner", axis=0)

            # Train-Test Split
            X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.2, random_state=42)

            # Fit OLS Model
            model = sm.OLS(y_train, X_train).fit()
            R2 = round(model.rsquared, 3)
            adj_R2 = round(model.rsquared_adj, 3)
            
            # Store Results
            results_list.append({
                'File': file_name,
                'Dependent Variable': column,
                'R-squared': R2,
                'Adjusted R-squared': adj_R2
            })

            # === Fancy Plots ===
            fig, axs = plt.subplots(2, 2, figsize=(14, 12))  # Create 2x2 grid for fancy plots

            # **1️⃣ Residual Plot (Detects heteroscedasticity)**
            sns.residplot(x=model.fittedvalues, y=y_train - model.fittedvalues, lowess=True,
                          ax=axs[0, 0], color="blue", scatter_kws={'alpha': 0.6})
            axs[0, 0].axhline(y=0, color="red", linestyle="--", linewidth=2)
            axs[0, 0].set_title("Residual Plot", fontsize=16)
            axs[0, 0].set_xlabel("Fitted Values")
            axs[0, 0].set_ylabel("Residuals")

            # **2️⃣ QQ Plot (Normality of residuals)**
            stats.probplot(model.resid, dist="norm", plot=axs[0, 1])
            axs[0, 1].set_title("QQ Plot (Residuals)", fontsize=16)

            # **3️⃣ Prediction vs. Actual Plot**
            sns.scatterplot(x=model.fittedvalues, y=y_train, ax=axs[1, 0], alpha=0.6, color='green', edgecolor='black')
            axs[1, 0].plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], color="red", linestyle="--", linewidth=2)
            axs[1, 0].set_title("Predicted vs Actual", fontsize=16)
            axs[1, 0].set_xlabel("Predicted Values")
            axs[1, 0].set_ylabel("Actual Values")

            # **4️⃣ Variance Inflation Factor (VIF) Heatmap (Detects Multicollinearity)**
            vif_data = pd.DataFrame()
            vif_data["Feature"] = X_filtered.columns
            vif_data["VIF"] = [variance_inflation_factor(X_filtered.values, i) for i in range(X_filtered.shape[1])]
            sns.heatmap(vif_data.set_index("Feature").T, annot=True, cmap="coolwarm", ax=axs[1, 1])
            axs[1, 1].set_title("Multicollinearity (VIF Heatmap)", fontsize=16)

            # Save all fancy plots into a PDF
            fancy_plots_filename = prefex + "Multiple_Linear_Regression_Results/Fancy_Plots_" + analysistype + ".pdf"
            with PdfPages(fancy_plots_filename) as pdf:
                pdf.savefig(fig, bbox_inches='tight', dpi=400)
                plt.close(fig)
            
            # === 3D Regression Surface Plot (Optional) ===
            if X_filtered.shape[1] > 2:
                fig = plt.figure(figsize=(10, 8))
                ax = fig.add_subplot(111, projection='3d')
                ax.scatter(X_filtered.iloc[:, 1], X_filtered.iloc[:, 2], y_filtered, color="b", alpha=0.6)
                ax.set_xlabel(X_filtered.columns[1])
                ax.set_ylabel(X_filtered.columns[2])
                ax.set_zlabel(column)
                ax.set_title("3D Regression Plot")

                # Save 3D plot
                three_d_plot_filename = prefex + "Multiple_Linear_Regression_Results/3D_Regression_" + analysistype + ".pdf"
                with PdfPages(three_d_plot_filename) as pdf:
                    pdf.savefig(fig, bbox_inches='tight', dpi=400)
                    plt.close(fig)

    # Convert Results to DataFrame
    results_df = pd.DataFrame(results_list)

    # === R² Heatmap ===
    r2_matrix = results_df.pivot(index='File', columns='Dependent Variable', values='R-squared')
    plt.figure(figsize=(10, 6))
    sns.heatmap(r2_matrix, annot=True, cmap="coolwarm", linewidths=0.5, fmt=".3f")
    plt.title("R-Squared Heatmap", fontsize=18)
    
    # Save Heatmap
    heatmap_filename = prefex + "Multiple_Linear_Regression_Results/R2_Heatmap.pdf"
    plt.savefig(heatmap_filename, dpi=400)
    plt.close()

    # Save results to CSV
    results_csv = prefex + "Multiple_Linear_Regression_Results/Regression_Results_" + analysistype + ".csv"
    results_df.to_csv(results_csv, index=False)

    return results_df.to_numpy().tolist()

def main():
    selected_files = []
    for file in file_list:
        selected_files.append(file)
        if selected_files:
             Multiple_Linear_Regression_Combined(selected_files, file.split('.')[0].split('_')[-2] + " " + file.split('.')[0].split('_')[-1], prefex)

    print("Job done!")

if __name__=="__main__":
    main()


Job done!
